In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
os.chdir('/content')
!tar xpf /content/drive/MyDrive/images.tgz
!mv '/content/images' '/content/images_12-3-21'

In [3]:
os.chdir('/content/images_12-3-21')
os.rename('negative', '0')
os.rename('people', '1')
os.rename('stopsign', '2')

In [4]:
# remove stop signs
#!rm -rf '/content/images_12-3-21/2' 
# remove people
!rm -rf '/content/images_12-3-21/1'
!mv '/content/images_12-3-21/2' '/content/images_12-3-21/1'

In [5]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.applications import imagenet_utils
from sklearn.metrics import confusion_matrix
import itertools
import os
import shutil
import random
import matplotlib.pyplot as plt
%matplotlib inline
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

# Organize data into train, valid, test dirs
os.chdir('/content/images_12-3-21')
if os.path.isdir('train/0/') is False: 
  os.mkdir('train')
  os.mkdir('valid')

  for i in range(0, 2):
    shutil.move(f'{i}', 'train')
    os.mkdir(f'valid/{i}')

    valid_samples = random.sample(os.listdir(f'train/{i}'), 30)
    for j in valid_samples:
      shutil.move(f'train/{i}/{j}', f'valid/{i}')

train_path = '/content/images_12-3-21/train'
valid_path = '/content/images_12-3-21/valid'

train_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input).flow_from_directory(directory=train_path, target_size=(224,224), batch_size=10)
valid_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input).flow_from_directory(directory=valid_path, target_size=(224,224), batch_size=10)

mobile = tf.keras.applications.mobilenet.MobileNet()

#x = mobile.layers[-6].output
x = mobile.output

output = Dense(units=2, activation='softmax')(x)

model = Model(inputs=mobile.input, outputs=output)

#for layer in model.layers[:-23]:
#    layer.trainable = False

model.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(x=train_batches,
            steps_per_epoch=len(train_batches),
            validation_data=valid_batches,
            validation_steps=len(valid_batches),
            epochs=80,
            verbose=2
)


Found 4010 images belonging to 2 classes.
Found 60 images belonging to 2 classes.
17235968/17225924 [==============================] - 0s 0us/step


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Epoch 1/80
401/401 - 67s - loss: 0.6158 - accuracy: 0.9803 - val_loss: 0.5998 - val_accuracy: 1.0000 - 67s/epoch - 167ms/step
Epoch 2/80
401/401 - 47s - loss: 0.5644 - accuracy: 0.9933 - val_loss: 0.5528 - val_accuracy: 1.0000 - 47s/epoch - 118ms/step
Epoch 3/80
401/401 - 48s - loss: 0.5211 - accuracy: 0.9955 - val_loss: 0.5171 - val_accuracy: 1.0000 - 48s/epoch - 119ms/step
Epoch 4/80
401/401 - 47s - loss: 0.4793 - accuracy: 0.9950 - val_loss: 0.4826 - val_accuracy: 1.0000 - 47s/epoch - 116ms/step
Epoch 5/80
401/401 - 46s - loss: 0.4424 - accuracy: 0.9973 - val_loss: 0.4543 - val_accuracy: 1.0000 - 46s/epoch - 115ms/step
Epoch 6/80
401/401 - 45s - loss: 0.4127 - accuracy: 0.9978 - val_loss: 0.4293 - val_accuracy: 1.0000 - 45s/epoch - 113ms/step
Epoch 7/80
401/401 - 46s - loss: 0.3869 - accuracy: 0.9975 - val_loss: 0.4065 - val_accuracy: 1.0000 - 46s/epoch - 116ms/step
Epoch 8/80
401/401 - 46s - loss: 0.3634 - accuracy: 0.9985 - val_loss: 0.3853 - val_accuracy: 1.0000 - 46s/epoch - 114

In [6]:
model.save('/content/stop_dec_mobilenet', save_format='tf')
# Converting a SavedModel to a TensorFlow Lite model.
converter = tf.lite.TFLiteConverter.from_saved_model('/content/stop_dec_mobilenet')
tflite_model = converter.convert()
open("/content/stop_dec_mobilenet.tflite", "wb").write(tflite_model)

INFO:tensorflow:Assets written to: /content/stop_dec_mobilenet/assets


16909432

In [7]:
!mv /content/stop_dec_mobilenet.tflite /content/drive/MyDrive/stop_mobilenet4_full.tflite